# **Imports**

In [1]:
!pip install datasets
!pip install gensim nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [1]:
import nltk
from nltk.tokenize import word_tokenize
from datasets import load_dataset
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from collections import Counter

# **Load dataset**

In [2]:
dataset = load_dataset("ag_news")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


# **Preprocessing**

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
def preprocess(text):
    # توکنایز کردن متن
    words = word_tokenize(text.lower())
    # حذف stop words و کاراکترهای غیرمفید
    return [word for word in words if word.isalpha() and word not in stop_words]


In [5]:
processed_data = []
for text in dataset['train']['text']:
    processed_data.append(preprocess(text))


In [6]:
processed_data[1]

['carlyle',
 'looks',
 'toward',
 'commercial',
 'aerospace',
 'reuters',
 'reuters',
 'private',
 'investment',
 'firm',
 'carlyle',
 'group',
 'reputation',
 'making',
 'plays',
 'defense',
 'industry',
 'quietly',
 'bets',
 'another',
 'part',
 'market']

# **Word2vec Model**

In [7]:
model_w2v = Word2Vec(sentences=processed_data, vector_size=100, window=5, min_count=1, workers=4)


In [8]:
model_w2v.save("word2vec_agnews.model")

In [9]:
similar_words = model_w2v.wv.most_similar('business', topn=5)
print(similar_words)

[('buying', 0.6475400924682617), ('investor', 0.6269737482070923), ('businesses', 0.6195728182792664), ('acquisitions', 0.6082369685173035), ('sector', 0.6011502146720886)]


In [10]:
def vectorize_text(text, model_w2v):
    words = preprocess(text)
    word_vectors = [model_w2v.wv[word] for word in words if word in model_w2v.wv]
    if len(word_vectors) == 0:
        return np.zeros(model_w2v.vector_size)
    return np.mean(word_vectors, axis=0)

# تبدیل داده‌ها به وکتور
X = np.array([vectorize_text(text, model_w2v) for text in dataset['train']['text']])

# برچسب‌های کلاس (labels)
y = np.array(dataset['train']['label'])

# تقسیم داده‌ها به داده‌های آموزش و ارزیابی
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ساخت مدل کلاسبند
clf = LogisticRegression(max_iter=1000)

# آموزش مدل
clf.fit(X_train, y_train)

# پیش‌بینی بر روی داده‌های ارزیابی
y_pred = clf.predict(X_val)

# ارزیابی مدل
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8865


# **Embedding model**

In [ ]:
texts = dataset['train']['text']
labels = dataset['train']['label']

# پیش‌پردازش و ایجاد دیکشنری
def build_vocab(texts):
    counter = Counter()
    for text in texts:
        words = preprocess(text)  # تابع پیش‌پردازش شما
        counter.update(words)
    vocab = {word: idx + 2 for idx, (word, _) in enumerate(counter.items())}  # idx=0 برای padding، idx=1 برای unk
    vocab['<pad>'] = 0
    vocab['<unk>'] = 1
    return vocab

vocab = build_vocab(texts)

# تبدیل متن به دنباله اعداد
def text_to_sequence(text, vocab):
    words = preprocess(text)
    return [vocab.get(word, vocab['<unk>']) for word in words]

# تبدیل تمام متن‌ها به دنباله اعداد
sequences = [text_to_sequence(text, vocab) for text in texts]

# Padding برای یکسان کردن طول دنباله‌ها
max_len = max(len(seq) for seq in sequences)
padded_sequences = np.array([
    seq + [vocab['<pad>']] * (max_len - len(seq)) for seq in sequences
])

# تقسیم داده‌ها
X_train, X_val, y_train, y_val = train_test_split(
    padded_sequences, labels, test_size=0.2, random_state=42
)

# تبدیل به تنسورهای PyTorch
X_train_tensor = torch.LongTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_val_tensor = torch.LongTensor(X_val)
y_val_tensor = torch.LongTensor(y_val)

# ایجاد DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        # x shape: (batch_size, seq_len)
        embedded = self.embedding(x)  # (batch_size, seq_len, embed_dim)
        # میانگین گیری روی بعد seq_len
        pooled = torch.mean(embedded, dim=1)  # (batch_size, embed_dim)
        out = self.fc(pooled)  # (batch_size, num_classes)
        return out

# پارامترها
vocab_size = len(vocab)
embed_dim = 100  # هماندازه با وکتور Word2Vec
num_classes = len(set(labels))

# ایجاد مدل
model = TextClassifier(vocab_size, embed_dim, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
from tqdm import tqdm

num_epochs = 10

for epoch in range(num_epochs):
    model.train()

    # استفاده از tqdm برای نمایش پیشرفت در هر epoch
    train_loop = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", leave=False)

    for batch_X, batch_y in train_loop:
        optimizer.zero_grad()
        outputs = model(batch_X.to(device))
        loss = criterion(outputs, batch_y.to(device))
        loss.backward()
        optimizer.step()

        # نمایش loss در نوار پیشرفت
        train_loop.set_postfix(loss=loss.item())

    # ارزیابی روی داده validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor.to(device))
        _, val_preds = torch.max(val_outputs, 1)
        accuracy = accuracy_score(y_val_tensor.cpu().numpy(), val_preds.cpu().numpy())

        # چاپ دقت در هر epoch
        print(f"Epoch {epoch + 1}, Validation Accuracy: {accuracy:.4f}")

Epoch 1, Validation Accuracy: 0.8950


Epoch 2, Validation Accuracy: 0.9056


Epoch 3, Validation Accuracy: 0.9085


Epoch 4, Validation Accuracy: 0.9107


Epoch 5, Validation Accuracy: 0.9113


Epoch 6, Validation Accuracy: 0.9093


Epoch 7, Validation Accuracy: 0.9078


Epoch 8, Validation Accuracy: 0.9061


Epoch 9, Validation Accuracy: 0.9036


Epoch 10, Validation Accuracy: 0.9018
